Improvements
- additional artist
- better regex filter
- making function to filter spotify more readable
- User input programm flow

# Scrape Billboard Top 100

## Parse HTML Page

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://www.billboard.com/charts/hot-100'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
soup.select('span.chart-element__information__song')[1].text

'Leave The Door Open'

## Extract Data

In [4]:
#song_names
song_lst = []
song_names = soup.select('span.chart-element__information__song')

for song in song_names:
    song_lst.append(song.text)

In [5]:
#artist_names
artist_lst = []
artist_names = soup.select('span.chart-element__information__artist')

for artist in artist_names:
    artist_lst.append(artist.text)

In [6]:
#ranking
ranking_lst = []
ranking_names = soup.select('span.chart-element__rank__number')

for rank in ranking_names:
    ranking_lst.append(rank.text)

## Convert to DataFrame and save

In [7]:
billboard_top = pd.DataFrame({'rank':ranking_lst, 'artist': artist_lst, 'song': song_lst})

In [8]:
billboard_top.head()

,rank,artist,song
0,1,The Weeknd & Ariana Grande,Save Your Tears
1,2,Silk Sonic (Bruno Mars & Anderson .Paak),Leave The Door Open
2,3,Justin Bieber Featuring Daniel Caesar & Giveon,Peaches
3,4,Polo G,Rapstar
4,5,Dua Lipa Featuring DaBaby,Levitating


In [9]:
billboard_top.to_csv('billboard_top.csv')

# User input validation

In [10]:
import re

artist_name = input('Artist: ')
if not re.match(".{4,}", artist_name):
        print ("Please enter a valid artist")
        artist_name = "9999999999xxxx"
        
song = input('Song: ')
if not re.match(".{4,}", artist_name):
        print ("Please enter a valid artist")
        song = "9999999999xxxx"
print(hot_or_not(artist_name, song))

Artist: test
Song: test


NameError: name 'hot_or_not' is not defined

In [ ]:
def hot_or_not (artist, song):
    '''
    function determines if artist is in the scraped database
    '''
    contains_artist = billboard_top['artist'].str.contains(artist_name)
    contains_title = billboard_top['song'].str.contains(song)
    results = pd.DataFrame({'contains_artist': contains_artist, 'contains_title': contains_title, "match": contains_artist & contains_title})

    if results['match'].any():
        s = 'Hot'
    else:
        s = 'Not'
    return s

# Creating Dataframe from Spotify Wrapper

In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pandas as pd
from tqdm.notebook import tqdm

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="7ac25cd2df894d4587f18918f0776908",
                                                           client_secret="947331b0345946ef847d79f23802ee84"))

In [14]:
spotifydf = pd.DataFrame()
artistdf = pd.DataFrame()
track_lst = []
artist_1 = []
artist_2 = []

#looping through categories
try:
    for category in tqdm(sp.categories(limit=15)['categories']['items']):

        #looping through playlists, ignoring ids that do not exist
        try:
            for playlist in sp.category_playlists(category_id=category['id'], country=None, limit=15, offset=0)['playlists']['items']:

                #looping through tracks, ignoring ids that do not exist
                try:
                    for tracks in sp.playlist_tracks(playlist['id'])['items']:
                        ids = tracks['track']['id']

                        #adding rows of audio_features
                        spotifydf = spotifydf.append(sp.audio_features(tracks['track']['id']))

                        #lst of track names
                        track_lst.append(sp.track(ids)['name'])

                        #lst of artist 1
                        artist_1.append(sp.track(ids)['artists'][0]['name'])

                        #lst of artist 2
                        try: artist_2.append(sp.track(ids)['artists'][1]['name'])
                        except: artist_2.append(None)
                except:
                    print('Track-ID not found error')
                    pass
        except:
            print('Playlist-ID not found error')
            pass
                    
except requests.exceptions.ReadTimeout:
    print("ReadTimeout")

finally: 
    spotifydf['track_names'] = track_lst
    spotifydf['artist_1'] = artist_1
    spotifydf['artist_2'] = artist_2

  0%|          | 0/9 [00:00<?, ?it/s]

Track-ID not found error



100%|██████████| 15/15 [1:17:27<00:00, 309.80s/it]


In [15]:
len(spotifydf)

11604

In [16]:
# Extracting to CSV
spotifydf.to_csv('spotifydf.csv')